# ДЗ

###### ДЗ 1 Классификация Домашнее задание из двух частей. (после НГ)

База про спортсменов

*Креативная часть:* найти парамеры и комбинации их, которые будут увеличивать accuracy

*Техническая часть:* сделать под это всё удобные обёртки. Например, это функция, которая просто принимает список фичей, а на выходе показывает результат фита

In [1]:
import pandas as pd
import glob
import os
import tqdm
import gc
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import seaborn as sns
import numpy as np
from scipy.spatial import Voronoi, voronoi_plot_2d
import math
from sklearn import datasets, linear_model, model_selection
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix
from tqdm import tqdm

In [2]:
##Для отображения плотов на лету
%matplotlib inline

###### Загрузка данных и чистка

In [3]:
f = r'data\athletes_sochi.txt'
dfs = pd.read_csv(f).dropna()

In [4]:
dfs

,age,birthdate,gender,height,name,weight,gold_medals,silver_medals,bronze_medals,total_medals,sport,country
0,17,1996-04-12,Male,1.72,Aaron Blunck,68.0,0,0,0,0,Freestyle Skiing,United States
1,27,1986-05-14,Male,1.85,Aaron March,85.0,0,0,0,0,Snowboard,Italy
2,21,1992-06-30,Male,1.78,Abzal Azhgaliyev,68.0,0,0,0,0,Short Track,Kazakhstan
4,21,1992-07-30,Male,1.86,Adam Barwood,82.0,0,0,0,0,Alpine Skiing,New Zealand
5,21,1992-12-18,Male,1.75,Adam Cieslar,57.0,0,0,0,0,Nordic Combined,Poland
...,...,...,...,...,...,...,...,...,...,...,...,...
2851,28,1985-04-30,Male,1.93,Ziga Pavlin,98.0,0,0,0,0,Ice Hockey,Slovenia
2853,31,1982-12-05,Female,1.70,Zina Kocher,60.0,0,0,0,0,Biathlon,Canada
2854,28,1985-06-14,Female,1.68,Zoe Gillings,65.0,0,0,0,0,Snowboard,Great Britain
2856,22,1991-03-01,Male,1.76,Zongyang Jia,68.0,0,0,1,1,Freestyle Skiing,China


In [5]:
# по статистике в женском спорте больше медалей всех типов на одного спортсмена (возможно, из-за более низкой конкуренции)
# но особенно разница заметна по золотым медалям, которые в среднем даже больше, чем остальных - а у мужчин все наоборот
dfs.groupby('gender').mean('age')

,age,height,weight,gold_medals,silver_medals,bronze_medals,total_medals
gender,,,,,,,
Female,25.136646,1.676501,61.699793,0.036232,0.032091,0.033126,0.101449
Male,26.499009,1.809022,80.390615,0.025777,0.028420,0.028420,0.082617


###### Подготовка дополнительных полей

In [6]:
dfs["BMI"] = dfs["weight"] / (dfs["height"]**2)

###### Подготовим входные данные

* набор пар для входных векторов X: пары это вес и рост
* набор пар для выходных векторов Y: это значения 1 -- male, 0 -- female
* перемешать набор данных весь и взять 20% отложить как test, остальное будет train

In [7]:
alldata = dfs[ ['weight', 'height', 'BMI', 'gender', 'gold_medals'] ]
alldata

,weight,height,BMI,gender,gold_medals
0,68.0,1.72,22.985398,Male,0
1,85.0,1.85,24.835646,Male,0
2,68.0,1.78,21.461937,Male,0
4,82.0,1.86,23.702162,Male,0
5,57.0,1.75,18.612245,Male,0
...,...,...,...,...,...
2851,98.0,1.93,26.309431,Male,0
2853,60.0,1.70,20.761246,Female,0
2854,65.0,1.68,23.030045,Female,0
2856,68.0,1.76,21.952479,Male,0


In [8]:
alldata = alldata.sample(frac=1)

In [9]:
alldata

,weight,height,BMI,gender,gold_medals
2307,69.0,1.74,22.790329,Male,0
2784,86.0,1.92,23.328993,Male,0
779,61.0,1.68,21.612812,Female,0
1924,79.0,1.81,24.114038,Female,0
1576,103.0,1.83,30.756368,Male,0
...,...,...,...,...,...
1128,58.0,1.75,18.938776,Male,0
2227,88.0,1.85,25.712199,Male,0
1236,79.0,1.75,25.795918,Male,0
454,58.0,1.72,19.605192,Female,0


In [10]:
allinput = alldata[ ['weight', 'height', 'BMI', 'gold_medals'] ]
alloutput = alldata["gender"].apply(lambda x: int(x == "Male"))

###### Разделим наши выборки

In [11]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(allinput, alloutput, test_size=0.2)

Создаем модель

In [12]:
model = linear_model.LogisticRegression()

Обучаем модель

In [13]:
model.fit(X_train, Y_train)

LogisticRegression()

Смотрим на коэффициенты

In [14]:
model.coef_

array([[ 0.3683111 ,  0.1818947 , -0.80384308, -0.41937204]])

Предскажем пол

In [15]:
Y_test_predicted = model.predict(X_test)

In [16]:
Y_test_predicted

array([1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1,

###### Численная оценка качества алгоритма Accuracy, precision и recall



In [17]:
[tn, fp], [fn, tp] = confusion_matrix(Y_test, Y_test_predicted)
confusion_matrix(Y_test, Y_test_predicted)

array([[166,  22],
       [ 39, 269]], dtype=int64)

In [18]:
precision = tp / (tp + fp)
recall = tp / tp + fn

print("Precision: ", tp / (tp + fp))
print("Recall: ", tp / (tp + fn))
print("Accuracy: ", (tn + tp) / (tp + fp + tn + fn))
print("F1: ",  (precision * recall) / ((precision) + recall))

# предыдущие значения (до рассмотрения наличия медали):
# Precision:  0.8778877887788779
# Recall:  0.8896321070234113
# Accuracy:  0.8588709677419355
# F1:  0.8557910673732021

Precision:  0.9243986254295533
Recall:  0.8733766233766234
Accuracy:  0.8770161290322581
F1:  0.9035183474683014


In [19]:
def predict_sex(weight, height, gold_medals, model):
    BMI = weight / height**2
    sex = model.predict([[weight, height, gold_medals, BMI]])
    if sex[0] == 1:
        print('Кажется вы мужчина.')
    else:
        print('Похоже вы женщина.')

In [20]:
predict_sex(81, 1.87, 0, model)

Кажется вы мужчина.


C:\Users\Aleksandr\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


###### ДЗ 2 Кластеризация  (после НГ)

*Креативная часть:* Релизовать на питоне ДЗ от вашего преподователя по МО

*Техническая часть:* сделать под это всё удобные обёртки. Например, это функция, которая просто принимает список фичей, а на выходе показывает результат фита

In [21]:
# on hold